# Derma AI

## Define datesets and model

### Imports

In [1]:
import os
import time
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torchvision import datasets
from PIL import ImageFile
import torchvision.transforms as transforms
from tqdm import tqdm_notebook as tqdm


### Specify Data Loaders for the dataset

In [2]:
# number of subprocesses to use for data loading
num_workers = 16
# how many samples per batch to load
batch_size = 20
# image size (side of square)
image_size = 224

# set data paths
data_dir = "data/"
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

# define transformation pipelines
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomResizedCrop(image_size),
                                      transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                     ])

basic_transform = transforms.Compose([transforms.Resize(256),
                                       transforms.CenterCrop(image_size),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])
                                     ])

# define datasets
data = {
    "train": datasets.ImageFolder(train_dir, transform=train_transform),
    "valid": datasets.ImageFolder(valid_dir, transform=basic_transform),
    "test": datasets.ImageFolder(test_dir, transform=basic_transform)
}

# prepare data loaders
loaders = {
    "train": torch.utils.data.DataLoader(data["train"], batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True),
    "valid": torch.utils.data.DataLoader(data["valid"], batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True),
    "test": torch.utils.data.DataLoader(data["test"], batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)
}

print("Train data:\t", len(loaders["train"].dataset))
print("Valid. data:\t", len(loaders["valid"].dataset))
print("Test data:\t", len(loaders["test"].dataset))

Train data:	 2000
Valid. data:	 150
Test data:	 600


In [3]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda:
    print("CUDA is available")
else:
    print("CUDA NOT available.")

CUDA is available


In [10]:
# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self, input_size, output_nodes):
        super(Net, self).__init__()

        self.input_size = input_size
        self.output_nodes = output_nodes
        
        ## Define layers of a CNN
        
        # Size 224
        self.conv1_1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(64)
        
        # Size 112
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(128)
        
        # Size 56
        self.conv3_1 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(256)
        
        # Size 28
        self.conv4_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(512)

        
        # Fully connected layers, Input: 14
        self.fc1 = nn.Linear(14*14*512, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, output_nodes)
        
        self.fc1_bn = nn.BatchNorm1d(512)
        self.fc2_bn = nn.BatchNorm1d(256)
        self.fc3_bn = nn.BatchNorm1d(64)
        
        # Pooling function
        self.pool = nn.MaxPool2d(2,2)
        
        # activation and dropout function
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        ## Define forward behavior

        # feature layers
        x = self.conv1_bn(self.conv1_1(x))
        x = self.conv1_bn(self.conv1_2(x))
        x = self.pool(self.relu(x))
        x = self.conv2_bn(self.conv2_1(x))
        x = self.conv2_bn(self.conv2_2(x))
        x = self.pool(self.relu(x))
        x = self.conv3_bn(self.conv3_1(x))
        x = self.conv3_bn(self.conv3_2(x))
        x = self.conv3_bn(self.conv3_3(x))
        x = self.pool(self.relu(x))
        x = self.conv4_bn(self.conv4_1(x))
        x = self.conv4_bn(self.conv4_2(x))
        x = self.conv4_bn(self.conv4_3(x))
        x = self.conv4_bn(self.conv4_4(x))
        x = self.pool(self.relu(x))
        
        # flatten tensor
        x = x.view(x.size(0), -1)
        
        # classifier
        
        x = self.dropout(self.fc1_bn(self.relu(self.fc1(x))))
        x = self.dropout(self.fc2_bn(self.relu(self.fc2(x))))
        x = self.dropout(self.fc3_bn(self.relu(self.fc3(x))))
        x = self.fc4(x)
        
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model = Net(image_size, len(data["train"].classes))

# move tensors to GPU if CUDA is available, print out model summary
if use_cuda:
    model.cuda()
    summary(model.cuda(), (3, image_size, image_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
       BatchNorm2d-2         [-1, 64, 224, 224]             128
            Conv2d-3         [-1, 64, 224, 224]          36,928
       BatchNorm2d-4         [-1, 64, 224, 224]             128
              ReLU-5         [-1, 64, 224, 224]               0
         MaxPool2d-6         [-1, 64, 112, 112]               0
            Conv2d-7        [-1, 128, 112, 112]          73,856
       BatchNorm2d-8        [-1, 128, 112, 112]             256
            Conv2d-9        [-1, 128, 112, 112]         147,584
      BatchNorm2d-10        [-1, 128, 112, 112]             256
             ReLU-11        [-1, 128, 112, 112]               0
        MaxPool2d-12          [-1, 128, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         295,168
      BatchNorm2d-14          [-1, 256,

## Train model

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [13]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path, 
          max_epochs_not_improved = 10):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    
    # Quits training after valid_loss_min not improving after X consecutive epochs
    #max_epochs_not_improved = 5
    
    # running counter
    cnt_epochs_not_improved = 0
    
    for epoch in range(1, n_epochs+1):
        # timer for epoch
        start = time.time()
        
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # backward pass: compute gradient
            loss.backward()
            # update parameters
            optimizer.step()
            # update running training loss
            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)
            #train_loss += loss.item()
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            ## update the average validation loss
            
            # forward pass
            output = model(data)
            # calculate the loss
            loss = criterion(output, target)
            # update running validation loss
            valid_loss += (1 / (batch_idx + 1)) * (loss.data - valid_loss)
            #valid_loss += loss.item()
            
        #train_loss = train_loss/len(loaders["train"])
        #valid_loss = valid_loss/len(loaders["valid"])

        # print training/validation statistics 
        print('Epoch: {}\tTraining Loss: {:.6f}\tValidation Loss: {:.6f}\tTime: {:.3f} seconds'
              .format(epoch, 
                      train_loss,
                      valid_loss,
                      time.time() - start
                     ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased. Saving model...')
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            cnt_epochs_not_improved = 0
        else:
            # Adds to count if model didn't improve
            # Quits training if threshold reached
            cnt_epochs_not_improved += 1
            if(cnt_epochs_not_improved == max_epochs_not_improved):
                print("Validation loss not decreasing anymore. Training stopped.")
                break

    # return trained model
    return model

In [14]:
# train the model
model = train(50, loaders, model, optimizer, 
                      criterion, use_cuda, 'model_derma.pt')

Epoch: 1	Training Loss: 1.039518	Validation Loss: 1.063918	Time: 49.801 seconds
Validation loss decreased. Saving model...
Epoch: 2	Training Loss: 0.889277	Validation Loss: 1.097090	Time: 49.782 seconds
Epoch: 3	Training Loss: 0.841708	Validation Loss: 1.099353	Time: 48.868 seconds
Epoch: 4	Training Loss: 0.798273	Validation Loss: 1.125645	Time: 48.032 seconds
Epoch: 5	Training Loss: 0.792644	Validation Loss: 1.113659	Time: 47.052 seconds
Epoch: 6	Training Loss: 0.800695	Validation Loss: 1.171663	Time: 50.529 seconds
Epoch: 7	Training Loss: 0.785728	Validation Loss: 1.114463	Time: 49.463 seconds
Epoch: 8	Training Loss: 0.777107	Validation Loss: 1.109518	Time: 48.473 seconds
Epoch: 9	Training Loss: 0.773456	Validation Loss: 1.175651	Time: 47.600 seconds
Epoch: 10	Training Loss: 0.770419	Validation Loss: 1.154363	Time: 48.065 seconds
Epoch: 11	Training Loss: 0.765249	Validation Loss: 1.211368	Time: 49.334 seconds
Validation loss not decreasing anymore. Training stopped.


In [15]:
# load the model that got the best validation accuracy
model.load_state_dict(torch.load('model_derma.pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [16]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            model.cuda()
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\tTest Accuracy: {:.1f} % ({}/{})'
              .format(test_loss, 100. * correct / total, int(correct), int(total)))

# call test function    
test(loaders, model, criterion, use_cuda)



Test Loss: 0.968527	Test Accuracy: 65.5 % (393/600)
